In [ ]:
!pip install seaborn --upgrade

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import sklearn as sk
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import itertools
sns.set_theme(style="whitegrid")

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing the training data

We import the titanic training data from `train.csv`.

In [ ]:
ROOT = '/kaggle/input/'
DATADIR = 'titanic/'

titanic = pd.read_csv(ROOT+DATADIR+'train.csv')
titanic.head()

# Descriptive analysis

We should examine our data for a moment and get an idea of its structure. We have 12 features and , with a mix of numerical and categorical data. If we want to perform any kind of numerical analysis involving the categorical features, we will probably need to transform it using e.g. a label encoder. 

We must also account for missing features in a record. We can use an imputer to fill NaN values with some value that we hope will not influence the statistics of the dataset; our filling strategy depends on how are data is distributed in that feature (skewed left, skewed right, centered on mean value, long tail, etc). Note that if the feature is categorical, we must convert the feature to numeric before we can impute any NaN values (if we wish to define statistical properties for that feature).

Let's get the descriptive statistics about the numerical data. We will ignore the categorical data for now.

In [ ]:
titanic.describe()

We have 891 observations and 7 numerical features. 
- `PassengerId` contains a unique identifier for each passenger on Titanic. 
- `Survived` contains the information that we wish to predict using some model of our choice or design. According to this dataset, only 38% of Titanic passengers survived the sinking. 
- `Pclass` is the ticket class. 
- `SibSp` is the number of siblings+spouses aboard the Titanic, for each unique PassengerId (each passenger has some number of siblings and/or a spouse with them) 
- `Age` is age
- `Parch` is the number of children+parents aboard the Titanic, for each unique PassengerId (similar to `SibSp`)
- `Fare` is the ticket cost (assume the currency is standardized, considering that the Titanic embarked from ports in France, England, and New Zealand)

### Cleaning the data

We notice that `Age` is missing 177 entries to the descriptive statistics, which suggests that our dataset may contain null or missing information. Let's first get an idea of exactly how much information is missing from all columns.

In [ ]:
titanic.isnull().sum()

We observe that `Cabin` contains 687 null values and `Embarked` contains 2 null values. Although cabin might be an interesting feature to study related to Titanic survivably (perhaps passengers in cabins closest to the escape boats were more likely to survive?), let's elminate it from our discussions in this report since we are losing over half of the `Cabin` information to null values.

We might hypothesize that `Age` plays a non-neglible role in Titanic survivability; therefore, rather than come up with a fill strategy, since `Age` may be an important predictor, we would be better off removing these rows completely from our training steps.

We could replace the 2 null values in `Embarked` with a fourth category, such as U for Unknown.

Let's drop the rows with null `Age` values, drop the `Cabin` column, and perform the suggested replacement in `Embarked`.

In [ ]:
titanic_dropped_na = titanic.dropna(subset=['Age']).drop(columns=['Cabin'])
titanic_dropped_na['Embarked'] = titanic_dropped_na['Embarked'].fillna('U')
titanic_dropped_na.isnull().sum()

We see that all null values have been eliminated from the dataset.

### Feature analysis

Next, we wish to examine the distributions of each feature. We can plot bar charts (histograms) of categorical feature, and use a kernel density estimator to approximately plot the distribution in each numerical feature. We omit the `Name` and `Ticket` columns from these observations, because these can be tied into the unique identifier provided by `PassengerId`.

In [ ]:
numerical_cols = ['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
categoric_cols = ['Sex', 'Embarked']
relevant_cols = numerical_cols + categoric_cols

# ------- FOR SEABORN 0.10.0 ------------------------------------------
# fig, axes = plt.subplots(3, 3, figsize=(16,8))
# fig.tight_layout(h_pad=5, w_pad=5)
# axes_points = list(set(itertools.combinations([0,1,2,0,1,2],2)))

# for i in range(len(numerical_cols)):
#     column = numerical_cols[i]
#     ax = axes_points[i]
#     sns.displot(data=titanic_dropped_na, x=column, ax=axes[ax], kde=True)
    
# fig, axes = plt.subplots(1, 2, figsize=(8,4))
# fig.tight_layout(h_pad=5, w_pad=5)

# for i in range(len(categoric_cols)):
#     column = categoric_cols[i]
#     ax = axes_points[i]
#     sns.countplot(x=column, data=titanic_dropped_na, ax=axes[i])
# ----------------------------------------------------------------------

fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(16,8))
fig.tight_layout(h_pad=5, w_pad=5)
for i, column in enumerate(relevant_cols):
    sns.histplot(titanic_dropped_na[column],ax=axes[i//3,i%3])

Eventually, we will probably want these numerical data scaled and normalized. Many prediction models, such as SVM classifiers, work best when using data transformed in this way. If we wish to use the categorical data for any sort of quantitative analyses, we will need to use some sort of encoder to transform the string variables in continuous variables.

We will continue this discussion in the exploratory analysis section below.

# Exploratory analysis

The goal is to predict which passengers survive. We need to assess which variables in our dataset are most correlated with the target variable, i.e. the column `Survived`. We can visualize correlations in our dataset by 1) obtaining the correlation matrix of the entire dataset and then 2) plotting the correlation matrix as a heatmap. Pandas and Seaborn have tools that let us complete these tasks.

Small aside on the correlation matrix. The correlation between two random variables $X$ and $Y$, as measured by the Pearson correlation coefficient, can be written as:

$$\text{corr}(X,Y) = \frac{E[XY] - E[X]E[Y]}{\sigma_X \sigma_Y}$$

where $E[X]$ is the *expected value* of $X$ and $\sigma_X$ is the standard deviation. Note that $\sigma_X = \sqrt{E[X^2] - E[X]^2}$. In a practical sense, the *expected value* is equivalent to the arithmetic mean. Then, the Pearson correleation coefficient measures the amount of *linear* dependence between the two random varibles $X$ and $Y$. A value of $\text{abs}\left(\text{corr}(X,Y)\right) \rightarrow 1$ implies that variation in the product $XY$ can be explained by the product of their variations, $\sigma_X \sigma_Y$. The sign implies the direction of the linear relationship (positive/negative slope). For $N$ random variables $X_1, X_2, \dots X_N$, we can construct a *correlation matrix* $\mathbf{M}$, with elements $\mathbf{M}_{ij} = \text{corr}(X_i,X_j)$, to visualize correlations between each pair of random variables. Of course, we can summarily ignore the diagonal elements, since we already know each random variable will be correlated with itself.

In [ ]:
titanic_corr = titanic_dropped_na.corr() # get the correlation matrix

fig = plt.figure(figsize=(16,8))
sns.heatmap(titanic_corr, annot=True) # plot the correlation matrix as a heatmap, annotating each cell with the correlation coefficient

We examine the correlation matrix for dependence of survival on our predictors, which are PassengerId, Pclass, Age, SibSp, Parch, and Fare. Pclass and Fare are the most correlated with `Survived`, so we may choose to treat these variables with importance.

Out of curiousity, let's perform a one-hot encoding on the categorical data, so that we might see how port of embarkation might affect survivability.

In [ ]:
one_hot_embark = pd.get_dummies(titanic_dropped_na.Embarked, prefix = 'Embarked')
titanic_dropped_na_join = titanic_dropped_na.join(one_hot_embark)
titanic_dropped_na_join

In [ ]:
columns = ['Embarked_C', 'Embarked_Q', 'Embarked_S', 'Embarked_U']
ports = ['Cherbourg', 'Queenstown', 'Southampton', 'Unknown']

total_passengers = titanic_dropped_na_join.shape[0]
for i, name in enumerate(ports):
    total_pass_from_port = titanic_dropped_na_join[columns[i]].sum()
    perc_pass_from_port = total_pass_from_port/total_passengers*100

    print("{:3d} passengers ({:5.2f}% of total passengers) embarked from {:s}".format(total_pass_from_port, perc_pass_from_port, name))

And now we can take another look at the correlation matrix.

In [ ]:
titanic_corr = titanic_dropped_na_join.corr() # get the correlation matrix

fig = plt.figure(figsize=(16,8))
sns.heatmap(titanic_corr, annot=True) # plot the correlation matrix as a heatmap, annotating each cell with the correlation coefficient

`Embarked_C` and `Embarked_S` are almost as correlated with `Survived` as `Fare`. There are only two data points present in `Embarked_U`, so we cannot say much about its relationship to `Survived`. Let's also look at how many passengers survived from each port of embarkation. This may explain some of the correlation.

In [ ]:
columns = ['Embarked_C', 'Embarked_Q', 'Embarked_S', 'Embarked_U']
ports = ['Cherbourg', 'Queenstown', 'Southampton', 'Unknown']

total_survived = titanic_dropped_na_join.Survived.sum()
for i, name in enumerate(ports):
    total_surv_from_port = titanic_dropped_na_join[titanic_dropped_na_join['Survived'] > 0][columns[i]].sum()
    perc_surv_from_port = total_surv_from_port/total_survived*100

    print("{:3d} survivors ({:5.2f}% of total survivors) embarked from {:s}".format(total_surv_from_port, perc_surv_from_port, name))

Now the correlation makes sense. The largest proportion of passengers embarked from Southampton, so, of course, the largest proportion of survivors embarked from Southampton. The proportion of survivors from each port also seems to resemble the proportion of passengers each port (within a few percent), which suggests that, within a given port of embarkation, a passenger was no more or less likely to survive than a passenger from a different port of embarkation. That is, each passenger had an equal chance to survive based on the port they embarked from. However, from a purely prediction standpoint, picking a passenger at random from the population, we could say that `Embarked` is a reasonable predictor for survivability. We will include the one-hot embarked in our prediction models.

Let's plot our data on a plane to see if we can observe any obvious clustering that might affect which models we decide to train in order to make predictions. We can plot our two most promising features, `Pclass` and `Fare`, against each other, with `Survived` on a color axis, to gain some qualitative insight on how easily we can find a separation plane between "survived" and "did not survive".

In [ ]:
fig = plt.figure(figsize=(8,8))
sns.scatterplot(data=titanic_dropped_na_join, x='Pclass', y='Fare', hue='Survived')

Visually, we run into some problems here because `Pclass` is not technically a continuous variable; it is a label that represents the socio-economic "class" of the passenger for which the ticket was purchased; however, a simple analogy would be to compare `Pclass` to "business" and "coach" on a plane (ignoring the fact that `Fare` would not vary so significantly in this case, since in plane ticket class prices are fixed). We do gain some qualitative insight from this strip plot: as far as clustering goes, the largest number of survivors is clustered in the 1st class ticket category. This observation motivates our use of `Pclass` as a predictor in our models.

Let's also examine `Fare` vs `Age` for an example of how we might analyze clustering for two continuous features.

In [ ]:
fig = plt.figure(figsize=(8,8))
sns.scatterplot(data=titanic_dropped_na_join, x='Fare', y='Age', hue='Survived')

`Age` is one of our weaker predictors for `Survived`, and we can see why. When we plot `Age` in a plane with `Fare` (one of our stronger predictors), no clear separation boundary is obvious between the two classes of `Survived`.

# Predictive analysis 
 
Next we will construct several models for predicting survivablility, and choose which one performs with the highest test case accuracy. We can build a model-testing pipeline using tools provided by the `sklearn` package.

Here we list the models we intend to test. Generally speaking, we can present the problem of predicting survivability as a classification problem. The Survived column contains binary (categorical) data, i.e. 1 if the passenger survived and 0 if the passenger did not survive. `sklearn` implements several classifiers. We understand that the goal of every classifier is to find a hyperplane that separates our data into "survived" and "did not survive".

Models:
1. SVC
    - SVMs attempt to find a hyperplane that separates categorical data by, essentially, maximizing the width between two categories of data
1. KNeighborsClassifier
    - "Majority voting" scheme where each data point is classified based on the labels of the $k$-nearest neighbors in a neighborhood around that point
1. RandomForestClassifier
    - A "forest" of decision tree classifiers. In a decision tree classifier, each "branch" is a feature that modulates the probability that a certain classification is made; e.g. a flower with petal width greater than 5 cm is 70% likely to be class 1, and less than 5 cm is 30% likely to be class 2. These probabilities are tuned by exposing the tree (or forest) to many examples during training
1. MLPClassifier
    - Neural network classifier. Several layers of nodes (input, hidden, and output) are linked together by weights, which are tuned through backpropagation according to some cost function
    


In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

X_train = titanic_dropped_na_join[['Pclass', 'Fare', 'Embarked_C', 'Embarked_S']]

y_train = np.ravel(titanic_dropped_na_join[['Survived']])


classifiers = [MLPClassifier(), SVC(), RandomForestClassifier(), KNeighborsClassifier()]

for classifier in classifiers:
    model_pipe = Pipeline(steps=[('StandardScaler', StandardScaler()), ('classifier', classifier)])
    model_pipe.fit(X_train, y_train)
    print(classifier)
    print('model training score: {:0.5f}\n'.format(model_pipe.score(X_train, y_train)))

Naively training these classifiers with default arguments, we see that some classifiers do achieve reasonable accuracy. The "model score" is the cross-validated mean squared error of the predicted output.

This dataset is small enough that we might as well try to train our models with all the features we have available. 

In [ ]:
# drop all irrelevant categorical data
X_train = titanic_dropped_na_join.drop(columns=['Survived', 'Name', 'Ticket', 'Embarked', 'Sex'])

y_train = np.ravel(titanic_dropped_na_join[['Survived']])


classifiers = [MLPClassifier(), SVC(), RandomForestClassifier(), KNeighborsClassifier()]

for classifier in classifiers:
    model_pipe = Pipeline(steps=[('StandardScaler', StandardScaler()), ('classifier', classifier)])
    model_pipe.fit(X_train, y_train)
    print(classifier)
    print('model training score: {:0.5f}\n'.format(model_pipe.score(X_train, y_train)))

Generally, performance increases for all classifiers after including all features. The RandomForestClassifier performs best, but we should be cautious of perfect predictions; usually, this means the model will not generalize well to an isolated testing set (overfitting). We could one-hot encode `Sex` to see if that feature offers another performance increase.

Our next task will be to perform a GridSearch for each model, in an attempt to optimize the model parameters and squeeze out the best performance from each one. We also wish to find under which conditions each model performs the best on a *testing* set (using cross validation).

In [ ]:
from sklearn.model_selection import GridSearchCV

# drop all irrelevant categorical data
X_train = titanic_dropped_na_join.drop(columns=['Survived', 'Name', 'Ticket', 'Embarked', 'Sex'])
# X_train = titanic_dropped_na_join[['Pclass', 'Fare', 'Embarked_C', 'Embarked_S']]
y_train = np.ravel(titanic_dropped_na_join[['Survived']])

## Coarse grid search for each classifier

We execute a coarse grid search on a custom parameter grid for each classifier we wish to tune for performance. This grid search is intended to be a cursory "first pass" to find which parameter combinations are worth tuning in more detail, in order to gain the most accuracy from a particular model. The cross-validated score represents the mean accuracy achieved on five folds of cross validation. Each fold isolates 1/5 of the training data as a testing set (i.e. data the trained model has never seen) and trains on the remaining 4/5 of the data.

### MLPClassifier

GridSearch on MLPClassifier. Parameters tested:

- `hidden_layer_sizes`: shape of hidden layers in neural network
- `activation`: activation function for the hidden layer
- `solver`: weight optimization
- `alpha`: L2 regularization strength
- `learning_rate`: method to determine weight updates

In [ ]:

mlp_class = MLPClassifier(max_iter=500)
model_pipe = Pipeline(steps=[('StandardScaler', StandardScaler()), ('MLPClassifier', mlp_class)])
print(model_pipe.get_params().keys())
param_grid = {'MLPClassifier__hidden_layer_sizes' : [(10,), (10,10), (100,)],
              'MLPClassifier__activation' : ['logistic', 'tanh', 'relu'],
              'MLPClassifier__solver' : ['adam', 'sgd'],
              'MLPClassifier__alpha' : [1e-7, 1e-5,1e-1, 1],
              'MLPClassifier__learning_rate' : ['constant']
             }

search = GridSearchCV(model_pipe, param_grid, n_jobs=-1, verbose=2)
search.fit(X_train, y_train)

In [ ]:
mlp_search_data = pd.DataFrame(search.cv_results_)
mlp_search_data.sort_values('rank_test_score')

### SVC

GridSearch on Support Vector Classifier. Parameters tested:

- `C`: regularization strength (inverse to magnitude; squared L2 regularization)
- `kernel`: the kernel function defines "distance measure" between points in parameter space (remember, goal is to find greatest separation in data)
- `degree`: polynomial kernel function degree
- `gamma`:  normalization for RBF, poly, and sigmoid kernel functions
- `coef0`: constant term in poly and sigmoid kernel funtions0

In [ ]:
svc_class = SVC(tol=1e-10)
model_pipe = Pipeline(steps=[('StandardScaler', StandardScaler()), ('SVC', svc_class)])
print(model_pipe.get_params().keys())
param_grid = {'SVC__C' : [1e-3,1e-1, 1.0, 10, 100],
              'SVC__kernel' : ['linear', 'poly', 'rbf', 'sigmoid'],
              'SVC__coef0' : [-1, -1e-2, 0.0, 1e-2, 1],
              'SVC__degree' : [2,3,4],
              'SVC__gamma' : ['scale', 'auto']
             }

search = GridSearchCV(model_pipe, param_grid, n_jobs=-1, verbose=2)
search.fit(X_train, y_train)

In [ ]:
svc_search_data = pd.DataFrame(search.cv_results_)
svc_search_data.sort_values('rank_test_score')

### RandomForestClassifier

GridSearch on RandomForestClassifier. This is a complicated classifier with many parameters to test, dealing with tree depth, pruning thresholds, branching probabilities, etc. Parameters tested:

- `n_estimators`: number of decision trees
- `max_depth`: maximum depth of each tree in the forest
- `min_samples_split`: minimum number of samples required to split a node
- `max_features`: number of features for best split; each tree is randomly assigned this number of features

In [ ]:
rfc_class = RandomForestClassifier()
model_pipe = Pipeline(steps=[('StandardScaler', StandardScaler()), ('RandomForestClassifier', rfc_class)])
print(model_pipe.get_params().keys())
param_grid = {'RandomForestClassifier__n_estimators' : [10, 100, 1000],
              'RandomForestClassifier__max_depth' : [10, 50, 100, 1000],
              'RandomForestClassifier__min_samples_split':[2,3,10],
              'RandomForestClassifier__max_features' : ['auto', 'sqrt', 'log2', 0.5*X_train.shape[1]]
             }

search = GridSearchCV(model_pipe, param_grid, n_jobs=-1, verbose=2)
search.fit(X_train, y_train)

In [ ]:
rfc_search_data = pd.DataFrame(search.cv_results_)
rfc_search_data.sort_values('rank_test_score')

### KNeighborsClassifier

GridSearch on k-nearest neighbors classifier. Parameters tested:

- `n_neighbors`: number of neighbors to compare against
- `weights`: weight function
- `p`:  changes dimension of metric (default metric is Minkowski)

In [ ]:
knc_class = KNeighborsClassifier()
model_pipe = Pipeline(steps=[('StandardScaler', StandardScaler()), ('KNeighborsClassifier', knc_class)])
print(model_pipe.get_params().keys())
param_grid = {'KNeighborsClassifier__n_neighbors' : [2, 3, 5, 8, 10],
              'KNeighborsClassifier__weights' : ['uniform', 'distance'],
              'KNeighborsClassifier__p' : [1,2,3,4,5]
             }

search = GridSearchCV(model_pipe, param_grid, n_jobs=-1, verbose=2)
search.fit(X_train, y_train)

In [ ]:
knc_search_data = pd.DataFrame(search.cv_results_)
knc_search_data.sort_values('rank_test_score')

In [ ]:
mlp = mlp_search_data.sort_values('rank_test_score').reset_index()
svc = svc_search_data.sort_values('rank_test_score').reset_index()
rfc = rfc_search_data.sort_values('rank_test_score').reset_index()
knc = knc_search_data.sort_values('rank_test_score').reset_index()


best_scores = pd.DataFrame({'classifier' : ['MultilayerPerceptron','SupportVectorClassifier','RandomForestClassifier','KNeighborsClassifier'],
                            'best_mean_score' : [mlp['mean_test_score'][0], svc['mean_test_score'][0], rfc['mean_test_score'][0],knc['mean_test_score'][0]]
                           })

fig = plt.figure(figsize=(16,8))
sns.barplot(y='classifier', x='best_mean_score', data=best_scores)

The average cross-validated scores for each classifier tested are extremely similar, with each score approaching 75% accuracy. The fact that each classifier is so similar suggests we are feature-limited. We may need to perform some feature engineering to add additional features, and hopefully increase the performance of each classifier.

We're going to write some functions here to make the entire model training process a little bit cleaner. We are also going to do some feature engineering. We will one-hot encode `Sex`, as well as create a new column called `Fsize`. `Fsize` ("family size") will contain a sum of the `Sibsp` and `Parch` columns for each row. Hopefully, this will elevate the robustness of our model training and translate to better cross-validated prediction performance.

In [ ]:
"""
    Run a cross-validated grid search and return the GridSearchCV object.
    
    Arguments:
    classifier -- the model to be trained
    param_grid -- the parameter grid on which to perform the search
    X_train    -- feature training data
    y_train    -- target training data
    
    Returns:
    search -- GridSearchCV object
"""
def pipe_train_cv(classifier, param_grid, X_train, y_train):
    model_pipe = Pipeline(steps=[('StandardScaler', StandardScaler()), (str(classifier), classifier)])
    
    search = GridSearchCV(model_pipe, param_grid, n_jobs=-1, verbose=2)
    search.fit(X_train, y_train)
    
    return (str(classifier), search)
    
"""
    Get the best mean scores from a list of DataFrames containing GridSearchCV results.
    
    Arguments:
    class_data_arr -- list of pandas DataFrames from GridSearchCV().cv_results_
    name_arr       -- list of names corresponding to models in class_data_arr
    
    Returns:
    best_scores_df -- DataFrame containing best mean score and best parameters for each model
"""
def get_best_scores(class_data_arr, name_arr=None):
    
    if name_arr == None:
        name_arr = ["classifier{:02d}".format(i) for i in range(len(class_data_arr))]
    
    class_data_arr_sorted = class_data_arr
    
    # sort the dataframes in the list
    for i, df in enumerate(class_data_arr):
        class_data_arr_sorted[i] = df.sort_values('rank_test_score').reset_index()

    best_scores_df = pd.concat([pd.DataFrame([[df['mean_test_score'][0], df['params'][0]]],
                                             columns = ['best_mean_score', 'best_param']) 
                                for df in class_data_arr_sorted], ignore_index=True)
    
    best_scores_df['classifier'] = name_arr
    
    return best_scores_df

We're going to one-hot encode the `Sex` column.

In [ ]:
one_hot_sex = pd.get_dummies(titanic_dropped_na_join.Sex, prefix = 'Sex')
titanic_dropped_na_join = titanic_dropped_na_join.join(one_hot_sex)
titanic_dropped_na_join

We're going to create the column `Fsize`, which will be a sum of `SibSp` and `Parch`. This represents size of family aboard the Titanic.

In [ ]:
titanic_dropped_na_join['Fsize'] = titanic_dropped_na_join.SibSp+titanic_dropped_na_join.Parch
titanic_dropped_na_join

### Training models on the additionally feature-engineered dataset

In [ ]:
# dropping categorical data we won't be using in prediction models
X_train = titanic_dropped_na_join.drop(columns=['Survived', 'Name', 'Ticket', 'Embarked', 'Sex'])
y_train = np.ravel(titanic_dropped_na_join[['Survived']])

In [ ]:
# Grid search on MLPClassifier model
mlp_param_grid = {'MLPClassifier()__hidden_layer_sizes' : [(10,), (10,10), (100,)],
                  'MLPClassifier()__activation' : ['logistic', 'tanh', 'relu'],
                  'MLPClassifier()__solver' : ['adam', 'sgd'],
                  'MLPClassifier()__alpha' : [1e-7, 1e-5,1e-1, 1],
                  'MLPClassifier()__learning_rate' : ['constant'], 
                  'MLPClassifier()__max_iter' : [1000] }
mlp_name, mlp_search = pipe_train_cv(MLPClassifier(), mlp_param_grid, X_train, y_train)

# Grid search on SupportVectorClassifier model
svc_param_grid = {'SVC()__C' : [1e-3,1e-1, 1.0, 10, 100],
                  'SVC()__kernel' : ['linear', 'poly', 'rbf', 'sigmoid'],
                  'SVC()__coef0' : [-1, -1e-2, 0.0, 1e-2, 1],
                  'SVC()__degree' : [2,3,4],
                  'SVC()__gamma' : ['scale', 'auto'] }
svc_name, svc_search = pipe_train_cv(SVC(), svc_param_grid, X_train, y_train)

# Grid search on RandomForestClassifier model
rfc_param_grid = {'RandomForestClassifier()__n_estimators' : [10, 100, 1000],
                  'RandomForestClassifier()__max_depth' : [10, 50, 100, 1000],
                  'RandomForestClassifier()__min_samples_split':[2,3,10],
                  'RandomForestClassifier()__max_features' : ['auto', 'sqrt', 'log2', 0.5*X_train.shape[1]] }
rfc_name, rfc_search = pipe_train_cv(RandomForestClassifier(), rfc_param_grid, X_train, y_train)

# Grid search on KNeighborsClassifier model
knc_param_grid = {'KNeighborsClassifier()__n_neighbors' : [2, 3, 5, 8, 10],
                  'KNeighborsClassifier()__weights' : ['uniform', 'distance'],
                  'KNeighborsClassifier()__p' : [1,2,3,4,5] }

knc_name, knc_search = pipe_train_cv(KNeighborsClassifier(), knc_param_grid, X_train, y_train)

In [ ]:
name_arr = [mlp_name, svc_name, rfc_name, knc_name]
class_data_arr = [
                    pd.DataFrame(mlp_search.cv_results_),
                    pd.DataFrame(svc_search.cv_results_),
                    pd.DataFrame(rfc_search.cv_results_),
                    pd.DataFrame(knc_search.cv_results_)
                ]

best_scores_df = get_best_scores(class_data_arr, name_arr)
best_scores_df

### Best mean cross-validated score from grid search

In [ ]:
fig = plt.figure(figsize=(16,8))
sns.barplot(y='classifier', x='best_mean_score', data=best_scores_df)

### Performance increase from models trained on original dataset

In [ ]:
best_scores_df['best_mean_score_old'] = best_scores.best_mean_score
best_scores_df['perc_performance_increase'] = best_scores_df.apply(lambda row: abs(row.best_mean_score - row.best_mean_score_old)/row.best_mean_score_old*100, axis=1)

fig = plt.figure(figsize=(16,8))
sns.barplot(y='classifier', x='perc_performance_increase', data=best_scores_df)

# Predicting survivors on the Titanic

Finally, we're going to apply our best estimator to predict the survivors in `test.csv`. We will use the SVC, although this model performed only marginally better than RandomForestClassifier. Perhaps with even more tweaking, or additionaly feature engineering, we might get even greater performance out of all our models.

We will need to perform the same feature-engineering on the testing set that we did on the training set. This includes one-hot encoding `Sex` and `Embarked`, and creating the column `Fsize`. We will not need to drop any NA values since we only wish to predict results, not train any models.

In [ ]:
# Load the test data from test.csv
test_data = pd.read_csv(ROOT+DATADIR+'test.csv')

# Need dummy column for Embarked_U because no Embarked NAs exist in testing set 
test_data['Embarked_U'] = np.zeros(test_data.shape[0],dtype=int)

# One hot encode Embarked
one_hot_embark = pd.get_dummies(test_data.Embarked, prefix = 'Embarked')
test_data_join = test_data.join(one_hot_embark)

# One hot encode Sex
one_hot_sex = pd.get_dummies(test_data_join.Sex, prefix = 'Sex')
test_data_join = test_data_join.join(one_hot_sex)

# Create Fsize
test_data_join['Fsize'] = test_data_join.SibSp+test_data_join.Parch

test_data_join

In [ ]:
test_data_join.isnull().sum()

During the descriptive analysis, we dropped a number of rows from the training data that contained NAs. We will not be able to do this with our testing set, so we need a strategy to fill NAs in `Age`, for example. We can ignore NAs in `Cabin` since we will not be using this column to make a prediction. 

Let's fill all NAs in `Age` with the average age. The motivation behind this strategy is this: if family size is somehow correlated with age, we hope that inserting the average age of all passengers will not substantially influence the correlation.

There is also a single `Fare` NA value. The strategy doesn't matter too much since there's only one value to replace. We will replace with the average.

In [ ]:
test_data_join['Age'] = test_data_join['Age'].fillna(test_data_join['Age'].mean())
test_data_join['Fare'] = test_data_join['Fare'].fillna(test_data_join['Fare'].mean())
test_data_join

In [ ]:
# Drop irrelevant categorical columns from our testing set
X_test = test_data_join.drop(columns=['Name', 'Ticket', 'Embarked', 'Sex', 'Cabin'])
X_test.isnull().sum()

In [ ]:
# Drop irrelevant categorical columns from our testing set
X_test = test_data_join.drop(columns=['Name', 'Ticket', 'Embarked', 'Sex', 'Cabin'])

# Get best SVC estimator from grid search results
best_svc = svc_search.best_estimator_

# Get predictions
y_pred = best_svc.predict(X_test)

Now we construct a DataFrame from the predictions. We can save this DataFrame as a CSV and upload to the competition.

In [ ]:
prediction_df = pd.DataFrame(X_test['PassengerId'])
prediction_df['Survived'] = y_pred
prediction_df.to_csv('titanic_predictions.csv',index=False)

# Conclusions

The best cross-validated score we were able to achieve was 82% using a SVC model. This is reasonable, but a score greater than 90% is most desirable to be competitive. One major way we could probably increase performance is by including more rows of data in our training set; recall that we dropped all *rows* that contained NA values in `Cabin`. The motivation behind this step was that, if any correlation between `Survived` and `Cabin` (or any correlation between `Cabin` and a good predictor of `Survived`) exists, simply dropping the entire column of `Cabin` might remove this relationship entirely from our predictive models. However, any cost we pay for removing the `Cabin` column is probably less than the training performance we stand to gain from including an additional 127 observations. Such a study can be persued in the future.